In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [21]:
train = pd.read_csv("../data/train.csv")
store = pd.read_csv("../data/store.csv")

/Applications/anaconda3/envs/mini-comp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
merged_train_store = pd.merge(store, train, on='Store', how='left')

In [23]:
merged_train_store = merged_train_store.drop(['Customers', 'Open'], axis=1)

In [24]:
zero_sales_mask = (merged_train_store.Sales == 0) | merged_train_store.Sales.isnull()
nonzero_sales = merged_train_store.loc[~zero_sales_mask]

In [25]:
nonzero_sales = nonzero_sales.dropna(axis=0, subset=['CompetitionDistance','StateHoliday','SchoolHoliday','Promo'])

In [26]:
nonzero_sales.Date = pd.to_datetime(nonzero_sales.Date)
nonzero_sales['Month'] = nonzero_sales['Date'].dt.month
nonzero_sales['DayOfMonth'] = nonzero_sales['Date'].dt.day
nonzero_sales['Year'] = nonzero_sales['Date'].dt.isocalendar()['year']
nonzero_sales['DayOfWeek'] = nonzero_sales['Date'].dt.isocalendar()['day']
nonzero_sales['WeekOfYear'] = nonzero_sales['Date'].dt.isocalendar()['week']
#nonzero_sales = nonzero_sales.drop('Date', axis=1)

In [27]:
zero_impute_cols = ['Promo2SinceWeek', 'Promo2SinceYear']
string_impute_cols = ['PromoInterval']

In [28]:
zero_impute = 0
string_impute = 'unavailable'

In [29]:
nonzero_sales[zero_impute_cols] = nonzero_sales[zero_impute_cols].fillna(zero_impute, axis=0)
nonzero_sales[string_impute_cols] = nonzero_sales[string_impute_cols].fillna(string_impute, axis=0)

In [30]:
100 * nonzero_sales.isnull().sum() / len(nonzero_sales)

Store                         0.000000
StoreType                     0.000000
Assortment                    0.000000
CompetitionDistance           0.000000
CompetitionOpenSinceMonth    31.614279
CompetitionOpenSinceYear     31.614279
Promo2                        0.000000
Promo2SinceWeek               0.000000
Promo2SinceYear               0.000000
PromoInterval                 0.000000
Date                          0.000000
DayOfWeek                     0.000000
Sales                         0.000000
Promo                         0.000000
StateHoliday                  0.000000
SchoolHoliday                 0.000000
Month                         0.000000
DayOfMonth                    0.000000
Year                          0.000000
WeekOfYear                    0.000000
dtype: float64

In [12]:
nonzero_sales.CompetitionDistance.describe()

count    452400.000000
mean       5437.409505
std        7762.579772
min          20.000000
25%         710.000000
50%        2320.000000
75%        6890.000000
max       75860.000000
Name: CompetitionDistance, dtype: float64

In [13]:
nonzero_sales[nonzero_sales.CompetitionOpenSinceMonth.isnull()].CompetitionDistance.describe()

count    143023.000000
mean       5459.808632
std        7539.149415
min          20.000000
25%         620.000000
50%        2460.000000
75%        7910.000000
max       75860.000000
Name: CompetitionDistance, dtype: float64

In [14]:
nonzero_sales[~nonzero_sales.CompetitionOpenSinceMonth.isnull()].CompetitionDistance.describe()

count    309377.000000
mean       5427.054532
std        7863.715177
min          30.000000
25%         730.000000
50%        2230.000000
75%        6540.000000
max       58260.000000
Name: CompetitionDistance, dtype: float64

In [15]:
mean_impute_cols = ['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']

In [16]:
mean_impute = np.floor(nonzero_sales[mean_impute_cols].mean())

In [17]:
nonzero_sales[mean_impute_cols] = nonzero_sales[mean_impute_cols].fillna(mean_impute, axis=0)

In [18]:
100 * nonzero_sales.isnull().sum() / len(nonzero_sales)

Store                        0.0
StoreType                    0.0
Assortment                   0.0
CompetitionDistance          0.0
CompetitionOpenSinceMonth    0.0
CompetitionOpenSinceYear     0.0
Promo2                       0.0
Promo2SinceWeek              0.0
Promo2SinceYear              0.0
PromoInterval                0.0
Date                         0.0
DayOfWeek                    0.0
Sales                        0.0
Promo                        0.0
StateHoliday                 0.0
SchoolHoliday                0.0
Month                        0.0
DayOfMonth                   0.0
Year                         0.0
WeekOfYear                   0.0
dtype: float64

In [19]:
nonzero_sales.PromoInterval.unique()

array(['unavailable', 'Jan,Apr,Jul,Oct', 'Feb,May,Aug,Nov',
       'Mar,Jun,Sept,Dec'], dtype=object)